#### Notebook for verifying gradient calculation

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from wind2watts.fit.util import fit_to_df
from wind2watts.data.util import velocity, airspeed, haversine

In [81]:
df = pd.read_csv(
    "../../data/dataframes/strava_export/5423193666.csv", parse_dates=["timestamp"]
)

In [82]:
df.columns

Index(['altitude', 'cadence', 'compressed_speed_distance', 'cycle_length',
       'distance', 'enhanced_altitude', 'enhanced_speed', 'grade',
       'heart_rate', 'position_lat', 'position_long', 'power', 'resistance',
       'speed', 'temperature', 'time_from_course', 'timestamp', 'year',
       'month', 'day', 'hour', 'wind_speed', 'wind_direction'],
      dtype='object')

In [144]:
def smooth_gradient(
    lat: np.ndarray,
    long: np.ndarray,
    altitude: np.ndarray,
    gradient_window: int = 3,
    altitude_window: int = 5,
):
    """
    Args:
        lat: 1D Numpy array representing latitudes of the points.
        long: 1D Numpy array representing longitudes of the points.
        altitude: 1D Numpy array representing altitudes of the points.
        gradient_window: The size of the moving window for smoothing the calculated gradient. Default is 3.
        altitude_window: The size of the moving window for smoothing the altitude values before gradient calculation. Default is 5.

    Returns:
        gradient: A 1D numpy array representing the smoothed gradient.

    Note:
        This function relies on the haversine() function to calculate the distance between latitude/longitude points. In case of any division by zero during the gradient calculation, those values are replaced by NaN.
    """
    lat1 = lat[:-1]
    lon1 = long[:-1]

    lat2 = lat[1:]
    lon2 = long[1:]

    dx = haversine(lat1, lon1, lat2, lon2)
    dx[dx == 0] = np.nan

    smoothed_y = pd.Series(altitude).rolling(altitude_window).mean().values
    dy = smoothed_y[1:] - smoothed_y[:-1]

    gradient = pd.Series(dy / dx).rolling(gradient_window).mean().interpolate().values

    return gradient

In [145]:
lat = df["position_lat"].values
long = df["position_long"].values
altitude = df["altitude"].values

gradient = smooth_gradient(lat, long, altitude)

len(altitude), len(gradient)

(3607, 3606)

In [ ]:
lat1 = lat[:-1]
lon1 = long[:-1]

lat2 = lat[1:]
lon2 = long[1:]

dx = haversine(lat1, lon1, lat2, lon2)
dx = pd.Series(dx).rolling(5, center=True).mean().values

# Take a 3 point moving average of dx
# dx_w = (dx[:-2] + dx[1:-1] + dx[2:]) / 3
# # Handle divide by zero
# eps = 1e-6
# mask_w = (dx_w < eps)
# dx_w[mask_w] = np.nan

# mask = (dx < eps)
# dx[mask] = np.nan


# altitude = df['altitude'].values
# altitude = pd.Series(altitude).rolling(5, center=True).mean().values
# # Take a 3 point moving average of altitude
# #altitude = (altitude[:-2] + altitude[1:-1] + altitude[2:]) / 3

# dy = altitude[1:] - altitude[:-1]
# print(len(dx), len(dy))

# gradient_w = dy[2:] / dx_w
# gradient = dy / dx


# # # Plot non-nan values
# mask = ~np.isnan(gradient)

# #plt.plot(gradient[mask], label='gradient')
# plt.plot(gradient_w[mask_w], label='gradient_w')
# plt.legend()
# # plt.plot(altitude[:100])